Gonna just backtest a bollinger bands strategy, gonna make it so you can 

In [11]:
import yfinance as yf
import pandas as pd
import backtesting as bt

In [ ]:
data = yf.download("NVDA", start="2020-01-01", end="2024-01-01")

data.columns = data.columns.get_level_values(0)
data = data.reset_index()
data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
#data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
#data.reset_index(inplace=True)
#data.index.name = None
data.dropna(inplace=True)
data.head()

/var/folders/f1/v4_lp9ks0dj4__tzb7_dw4k00000gn/T/ipykernel_78198/1167348606.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("NVDA", start="2020-01-01", end="2024-01-01")
[*********************100%***********************]  1 of 1 completed


KeyError: "None of ['Price'] are in the columns"

In [79]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class BBandsStrategy(Strategy):
    n = 20  # Period for moving average
    k = 2   # Number of standard deviations for the bands

    def init(self):
        self.sma = self.I(SMA, self.data.Close, self.n)
        self.std = self.I(pd.Series.rolling, self.data.Close, window=self.n).std()
        self.upper_band = self.sma + self.k * self.std
        self.lower_band = self.sma - self.k * self.std

    def next(self):
        if crossover(self.data.Close, self.lower_band):
            self.buy()
        elif crossover(self.upper_band, self.data.Close):
            self.sell()



In [80]:

bt = Backtest(data, BBandsStrategy, cash=10000, commission=0.002)
stats = bt.run()
print(stats)

/var/folders/f1/v4_lp9ks0dj4__tzb7_dw4k00000gn/T/ipykernel_78198/1780380771.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(data, BBandsStrategy, cash=10000, commission=0.002)


RuntimeError: Indicator "rolling(C,20)" error. See traceback above.